calculate additional descriptors. you can also calculate these in the call to rdkit_molecular_descriptors and add them to the json

In [ ]:
%%timeit -n 1 -r 1

%%bigquery 

create or replace table `exports.smiles_fp_mol_descriptors_json` as
(
SELECT smiles,
`cheminformatics.rdkit_fingerprint`(smiles) as fingerprint_json,
`cheminformatics.rdkit_molecular_descriptors`(smiles) as molecular_descriptors_json,
`cheminformatics.rdkit_qed`(smiles) as rdkit_qed,
`cheminformatics.rdkit_smiles_to_inchi`(smiles) as inchi,
from `sample_data.smiles_bin_parted` 
)

Benchmark fingerprint creation across 10 million compounds

In [ ]:
%%timeit -n 2 -r 2

%%bigquery 

create or replace table `exports.smiles_fp_only_json` as
(
SELECT smiles,
`cheminformatics.rdkit_fingerprint`(smiles) as fingerprint_json,
from `sample_data.smiles_bin_parted` 
)

Benchmark fingerprint across 10 million compounds that haven't been partitioned

In [ ]:
%%timeit -n 1 -r 1

%%bigquery 

create or replace table `exports.smiles_fp_only_json` as
(
SELECT smiles,
`cheminformatics.rdkit_fingerprint`(smiles) as fingerprint_json,
from `sample_data.smiles` 
)

Why do I have to partition the dataset to get the remote functions executing at scale? Is it something about the ncbi table I started with or the way I generated the original smiles dataset?

Benchmark fingerprint creation across 1 Billion compounds

In [ ]:
%%timeit -n 1 -r 1

%%bigquery 

create or replace table `exports.savi_fp_only_json` as
(
SELECT product_smiles,
`cheminformatics.rdkit_fingerprint`(product_smiles) as fingerprint_json,
from `nih-nci-cbiit-chem-prod.savi.plus` 
)

First run of this actually took 13 minutes. The above 10 minute results are when the cloud functions are "warm". 

## Bio Example

In [ ]:
%%timeit -n 1 -r 5
%%bigquery df 

SELECT 
`cheminformatics.biopython_sequence_translate`("gtatctagaagaaagaacgatgcggctcccacgtgcaaactctctaagaattacactgtttgtccatgagcgcgctacaatgatatcggtccctcgcctccacaaccgggacttgctttatgcgggctccagctgattgagacatagtgagcctcacaactgcgtcgccttcacgccgcagctggagcggtaaaagtttaactattactacagtcgggtcatgactttgtccaggactacgcctatgggtcc") as amino_acid_literal,
`cheminformatics.biopython_sequence_translate_to_stop`("gtatctagaagaaagaacgatgcggctcccacgtgcaaactctctaagaattacactgtttgtccatgagcgcgctacaatgatatcggtccctcgcctccacaaccgggacttgctttatgcgggctccagctgattgagacatagtgagcctcacaactgcgtcgccttcacgccgcagctggagcggtaaaagtttaactattactacagtcgggtcatgactttgtccaggactacgcctatgggtcc") as amino_acid_translated;